In [357]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [358]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-qfsh1v_c
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-qfsh1v_c
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [359]:
%load_ext nvcc4jupyter

The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [360]:
!git clone https://github.com/nothings/stb.git
!cp stb/stb_image.h /usr/local/include/

fatal: destination path 'stb' already exists and is not an empty directory.


In [361]:
!pip install wget
!wget https://raw.githubusercontent.com/nothings/stb/master/stb_image_write.h

--2024-04-20 17:14:34--  https://raw.githubusercontent.com/nothings/stb/master/stb_image_write.h
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71221 (70K) [text/plain]
Saving to: ‘stb_image_write.h.10’

stb_image_write.h.1 100%[===================>]  69.55K  --.-KB/s    in 0.05s   

2024-04-20 17:14:35 (1.47 MB/s) - ‘stb_image_write.h.10’ saved [71221/71221]



# Final =========================>


In [362]:
import shutil
import os

# Path to the uploaded image file
image_path = './in_3_4/cat.jpg'  # Update with the path of your uploaded image

# Directory to save the copies
output_dir = './in_all/'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Number of copies to generate
num_copies = 50

# Generate copies
for i in range(num_copies):
    shutil.copy(image_path, output_dir + f'copy_{i}.jpg')

print(f'{num_copies} copies generated successfully.')

50 copies generated successfully.


In [363]:
%%writefile K1_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <iomanip>
#include <fstream>
#include <vector>
#define STB_IMAGE_IMPLEMENTATION
#include <stdio.h>
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"
#include <string>
#include <filesystem>
#include <cuda_runtime.h>
using namespace std;

__constant__ float mask[15*15];


// Kernel definition
__global__ void Imgcov3d(int batchSize, float *input, float* output ,unsigned int width, unsigned int height , int maskdim){

    int outRow = blockIdx.y * blockDim.y + threadIdx.y;
    int outCol = blockIdx.x * blockDim.x + threadIdx.x;
    int imageIndex = blockIdx.z * blockDim.z + threadIdx.z;

    if(outRow < height && outCol < width && imageIndex < batchSize) {

      float sum = 0;

      for(int maskdep = 0; maskdep < 3; ++maskdep){
        for(int maskRow = 0; maskRow < maskdim; ++maskRow){
           for(int maskCol = 0; maskCol < maskdim; ++maskCol){

                  int inRow = outRow -(maskdim/2) + maskRow;
                  int inCol = outCol -(maskdim/2) + maskCol;

                  if(inRow < height && inRow >= 0 && inCol < width && inCol >= 0){
                      sum += mask[maskRow * maskdim + maskCol]
                      *input[imageIndex * 3 * width * height + maskdep * width * height + inRow * width + inCol];
                  }
             }
          }
        }

        if(sum < 0){
          sum = 0;
        } else if (sum > 255){
          sum = 255;
        }

        output[imageIndex * width * height + outRow * width + outCol] = sum;
    }

}

int main(int argc, char *argv[]) {

    string folderPath = argv[1];
    string out_path = argv[2];
    int batch_size = stoi(argv[3]);
    string mask_path = argv[4];

    int maskdim;

    // Open the file for reading
    std::ifstream inputFile(mask_path);

    // Check if the file is opened successfully
    if (!inputFile.is_open()) {
        std::cerr << "Error opening file: " <<mask_path  << std::endl;
        return 1; // Exit with an error code
    }

     inputFile >> maskdim ;
     // cout<<"maskdim "<<maskdim<<endl;

    float *h_A, *h_B,*R,*G,*B;
    int rows1,cols1, comp;

    float *h_images;
    int img_index = 0;

    for (const auto& entry : std::filesystem::directory_iterator(folderPath)) {

        if (entry.is_regular_file()) {
            std::string filePath = entry.path().string();

            unsigned char *data = stbi_load(filePath.c_str(), &cols1, &rows1, &comp, 0);

            if(img_index == 0){
              h_images = (float*)malloc(sizeof(float) * batch_size * cols1 * rows1 * comp);
            }

            if (data) {

                /*printf("cols %d\n", cols1);
                printf("rows %d\n", rows1);
                printf("image %s\n", filePath.c_str());
                */

                h_A = (float*)malloc(sizeof(float) *  rows1 * cols1* comp);
                R = (float*)malloc(sizeof(float) *  rows1 * cols1);
                G = (float*)malloc(sizeof(float) *  rows1 * cols1);
                B = (float*)malloc(sizeof(float) *  rows1 * cols1);

                int k = 0;
                for(int i=0;i<rows1 * cols1* comp;i=i+3)
                {
                  R[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=1;i<rows1 * cols1* comp;i=i+3)
                {
                  G[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=2;i<rows1 * cols1* comp;i=i+3)
                {
                  B[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }


                memcpy(h_A, R, rows1*cols1 * sizeof(float));
                memcpy(h_A +rows1*cols1 , G, rows1*cols1 * sizeof(float));
                memcpy(h_A+rows1*cols1+rows1*cols1, B, rows1*cols1 * sizeof(float));

                memcpy(h_images + img_index * rows1 * cols1 * 3,                     R, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1,     G, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1 * 2, B, rows1 * cols1 * sizeof(float));

                img_index++;

            }
            else {
                // Failed to load the image
                std::cerr << "Failed to load image: " << filePath << std::endl;
            }

           }
      }


      float *d_images_temp = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1 * comp);
      for(int i=0; i<batch_size; i++){
        memcpy(d_images_temp + i * rows1 * cols1 *comp , h_images + i * rows1 * cols1 *comp, rows1 * cols1 *comp * sizeof(float));
      }


    float *d_images;
    cudaMalloc((void**)&d_images, batch_size * rows1 * cols1 * comp * sizeof(float));
    cudaMemcpy(d_images, d_images_temp, batch_size * rows1 * cols1 * comp * sizeof(float), cudaMemcpyHostToDevice);

    float *d_outs;
    cudaMalloc((void**)&d_outs, batch_size * rows1 * cols1 * sizeof(float));

    float *h_outs = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1);

    h_B =(float*)malloc(sizeof(float) * maskdim* maskdim);

    // Read mask elements
    for (int i = 0; i < maskdim; ++i) {
        for (int j = 0; j < maskdim; ++j) {

            int index =  i * maskdim + j;
            inputFile >>  h_B[index];

        }
    }

    // Copy data from host to device
    cudaMemcpyToSymbol(mask, h_B, maskdim * maskdim * sizeof(float), 0, cudaMemcpyHostToDevice);


    // Kernel invocation
    dim3 threadsPerBlock(16,16,1);
    dim3 numBlocks((cols1-1) / threadsPerBlock.x + 1, (rows1-1) / threadsPerBlock.y + 1, batch_size);
    Imgcov3d<<<numBlocks, threadsPerBlock>>>(batch_size,d_images, d_outs,cols1,rows1, maskdim);

    cudaMemcpy(h_outs, d_outs, batch_size * rows1 * cols1 * sizeof(float), cudaMemcpyDeviceToHost);

    unsigned char* h_char = (unsigned char*)malloc(sizeof(unsigned char) * rows1 * cols1);

    for(int k=0;k<batch_size;k++){

      for(int i=0;i<rows1*cols1;i++){
         h_char[i]=static_cast<unsigned char>(h_outs[k*rows1*cols1+i]);
      }
      string filename = "./"+out_path+"/"+"image" + std::to_string(k) + ".jpg";
      stbi_write_jpg(filename.c_str(), cols1, rows1, 1, h_char, 100);
    }

    // Free device memory
    cudaFree(d_images);
    cudaFree(d_outs);

    // Free host memory
    free(h_A);
    free(h_B);
    free(R);
    free(G);
    free(B);
    free(h_images);
    free(d_images_temp);
    free(h_outs);
    free(h_char);

  return 0;
}

Overwriting K1_2_31_2_33.cu


In [364]:
!chmod +x K1_2_31_2_33.cu
!nvcc  K1_2_31_2_33.cu -o K1_2_31_2_33

In [365]:
!nvprof ./K1_2_31_2_33 "in_3_4" "out1_255" 4 "mask_3.txt"

==45230== NVPROF is profiling process 45230, command: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_3.txt
==45230== Profiling application: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_3.txt
==45230== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   51.36%  152.23ms         1  152.23ms  152.23ms  152.23ms  [CUDA memcpy DtoH]
                   42.75%  126.73ms         2  63.364ms     704ns  126.73ms  [CUDA memcpy HtoD]
                    5.89%  17.457ms         1  17.457ms  17.457ms  17.457ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int)
      API calls:   60.13%  297.65ms         2  148.83ms  126.94ms  170.71ms  cudaMemcpy
                   39.08%  193.49ms         2  96.743ms  429.97us  193.06ms  cudaMalloc
                    0.65%  3.2319ms         2  1.6159ms  981.31us  2.2506ms  cudaFree
                    0.08%  408.46us         1  408.46us  408.46us  408.46us  cudaMemcpyToSymbol
                    0.04%

In [366]:
!nvprof ./K1_2_31_2_33 "in_3_4" "out1_255" 4 "mask_5.txt"

==45282== NVPROF is profiling process 45282, command: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_5.txt
==45282== Profiling application: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_5.txt
==45282== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   45.81%  137.54ms         2  68.769ms     704ns  137.54ms  [CUDA memcpy HtoD]
                   44.76%  134.39ms         1  134.39ms  134.39ms  134.39ms  [CUDA memcpy DtoH]
                    9.43%  28.299ms         1  28.299ms  28.299ms  28.299ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int)
      API calls:   60.43%  301.25ms         2  150.62ms  137.75ms  163.50ms  cudaMemcpy
                   38.98%  194.31ms         2  97.157ms  372.45us  193.94ms  cudaMalloc
                    0.47%  2.3593ms         2  1.1797ms  1.0200ms  1.3393ms  cudaFree
                    0.08%  384.16us         1  384.16us  384.16us  384.16us  cudaMemcpyToSymbol
                    0.03%

In [367]:
!nvprof ./K1_2_31_2_33 "in_3_4" "out1_255" 4 "mask_7.txt"

==45333== NVPROF is profiling process 45333, command: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_7.txt
==45333== Profiling application: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_7.txt
==45333== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   43.70%  135.13ms         1  135.13ms  135.13ms  135.13ms  [CUDA memcpy DtoH]
                   39.78%  123.01ms         2  61.505ms     703ns  123.01ms  [CUDA memcpy HtoD]
                   16.52%  51.099ms         1  51.099ms  51.099ms  51.099ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int)
      API calls:   61.90%  310.29ms         2  155.15ms  123.26ms  187.03ms  cudaMemcpy
                   37.51%  188.01ms         2  94.007ms  579.55us  187.43ms  cudaMalloc
                    0.44%  2.2249ms         2  1.1125ms  1.0154ms  1.2095ms  cudaFree
                    0.10%  500.29us         1  500.29us  500.29us  500.29us  cudaMemcpyToSymbol
                    0.03%

In [368]:
!nvprof ./K1_2_31_2_33 "in_3_4" "out1_255" 4 "mask_9.txt"

==45378== NVPROF is profiling process 45378, command: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_9.txt
==45378== Profiling application: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_9.txt
==45378== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   41.11%  138.92ms         1  138.92ms  138.92ms  138.92ms  [CUDA memcpy DtoH]
                   38.10%  128.73ms         2  64.363ms     672ns  128.73ms  [CUDA memcpy HtoD]
                   20.79%  70.236ms         1  70.236ms  70.236ms  70.236ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int)
      API calls:   63.10%  338.99ms         2  169.49ms  128.96ms  210.03ms  cudaMemcpy
                   36.36%  195.35ms         2  97.676ms  392.05us  194.96ms  cudaMalloc
                    0.42%  2.2527ms         2  1.1264ms  911.94us  1.3408ms  cudaFree
                    0.08%  410.72us         1  410.72us  410.72us  410.72us  cudaMemcpyToSymbol
                    0.03%

In [369]:
!nvprof ./K1_2_31_2_33 "in_3_4" "out1_255" 4 "mask_11.txt"

==45427== NVPROF is profiling process 45427, command: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_11.txt
==45427== Profiling application: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_11.txt
==45427== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   36.54%  133.31ms         1  133.31ms  133.31ms  133.31ms  [CUDA memcpy DtoH]
                   33.90%  123.69ms         2  61.845ms     704ns  123.69ms  [CUDA memcpy HtoD]
                   29.57%  107.89ms         1  107.89ms  107.89ms  107.89ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int)
      API calls:   66.49%  365.89ms         2  182.95ms  123.90ms  242.00ms  cudaMemcpy
                   32.83%  180.64ms         2  90.322ms  385.22us  180.26ms  cudaMalloc
                    0.59%  3.2381ms         2  1.6191ms  1.0198ms  2.2183ms  cudaFree
                    0.06%  329.95us         1  329.95us  329.95us  329.95us  cudaMemcpyToSymbol
                    0.0

In [370]:
!nvprof ./K1_2_31_2_33 "in_3_4" "out1_255" 4 "mask_13.txt"

==45476== NVPROF is profiling process 45476, command: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_13.txt
==45476== Profiling application: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_13.txt
==45476== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   35.27%  136.64ms         1  136.64ms  136.64ms  136.64ms  [CUDA memcpy DtoH]
                   32.78%  127.02ms         1  127.02ms  127.02ms  127.02ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int)
                   31.95%  123.77ms         2  61.887ms     736ns  123.77ms  [CUDA memcpy HtoD]
      API calls:   67.11%  388.67ms         2  194.33ms  124.00ms  264.67ms  cudaMemcpy
                   32.23%  186.63ms         2  93.317ms  403.85us  186.23ms  cudaMalloc
                    0.55%  3.1982ms         2  1.5991ms  947.99us  2.2502ms  cudaFree
                    0.06%  371.06us         1  371.06us  371.06us  371.06us  cudaMemcpyToSymbol
                    0.0

In [371]:
!nvprof ./K1_2_31_2_33 "in_3_4" "out1_255" 4 "mask_15.txt"

==45521== NVPROF is profiling process 45521, command: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_15.txt
==45521== Profiling application: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_15.txt
==45521== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   41.99%  186.83ms         1  186.83ms  186.83ms  186.83ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int)
                   30.52%  135.77ms         1  135.77ms  135.77ms  135.77ms  [CUDA memcpy DtoH]
                   27.49%  122.30ms         2  61.151ms     736ns  122.30ms  [CUDA memcpy HtoD]
      API calls:   70.58%  445.93ms         2  222.97ms  122.53ms  323.40ms  cudaMemcpy
                   28.98%  183.11ms         2  91.556ms  363.49us  182.75ms  cudaMalloc
                    0.33%  2.1120ms         2  1.0560ms  923.87us  1.1881ms  cudaFree
                    0.06%  387.40us         1  387.40us  387.40us  387.40us  cudaMemcpyToSymbol
                    0.0

In [372]:
!nvprof ./K1_2_31_2_33 "in_3_4" "out1_255" 4 "mask_edge_5.txt"

==45571== NVPROF is profiling process 45571, command: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_edge_5.txt
==45571== Profiling application: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_edge_5.txt
==45571== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   48.94%  144.89ms         1  144.89ms  144.89ms  144.89ms  [CUDA memcpy DtoH]
                   41.50%  122.85ms         2  61.423ms     704ns  122.85ms  [CUDA memcpy HtoD]
                    9.56%  28.299ms         1  28.299ms  28.299ms  28.299ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int)
      API calls:   61.46%  297.32ms         2  148.66ms  123.06ms  174.26ms  cudaMemcpy
                   37.93%  183.50ms         2  91.750ms  380.41us  183.12ms  cudaMalloc
                    0.48%  2.3030ms         2  1.1515ms  984.51us  1.3185ms  cudaFree
                    0.09%  417.68us         1  417.68us  417.68us  417.68us  cudaMemcpyToSymbol
               

In [373]:
!nvprof ./K1_2_31_2_33 "in_3_4" "out1_255" 4 "mask_3.txt"

==45626== NVPROF is profiling process 45626, command: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_3.txt
==45626== Profiling application: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_3.txt
==45626== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   48.81%  134.99ms         1  134.99ms  134.99ms  134.99ms  [CUDA memcpy DtoH]
                   44.88%  124.13ms         2  62.063ms     672ns  124.13ms  [CUDA memcpy HtoD]
                    6.31%  17.450ms         1  17.450ms  17.450ms  17.450ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int)
      API calls:   59.49%  277.63ms         2  138.81ms  124.36ms  153.26ms  cudaMemcpy
                   40.00%  186.70ms         2  93.349ms  398.91us  186.30ms  cudaMalloc
                    0.38%  1.7909ms         2  895.45us  593.52us  1.1974ms  cudaFree
                    0.08%  395.27us         1  395.27us  395.27us  395.27us  cudaMemcpyToSymbol
                    0.03%

In [374]:
!nvprof ./K1_2_31_2_33 "in_3_4" "out1_255" 4 "mask_15.txt"

==45675== NVPROF is profiling process 45675, command: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_15.txt
==45675== Profiling application: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_15.txt
==45675== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   40.24%  174.20ms         1  174.20ms  174.20ms  174.20ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int)
                   31.31%  135.54ms         1  135.54ms  135.54ms  135.54ms  [CUDA memcpy DtoH]
                   28.44%  123.12ms         2  61.562ms     736ns  123.12ms  [CUDA memcpy HtoD]
      API calls:   68.34%  433.90ms         2  216.95ms  123.36ms  310.54ms  cudaMemcpy
                   31.23%  198.29ms         2  99.144ms  366.07us  197.92ms  cudaMalloc
                    0.34%  2.1480ms         2  1.0740ms  940.08us  1.2080ms  cudaFree
                    0.06%  380.10us         1  380.10us  380.10us  380.10us  cudaMemcpyToSymbol
                    0.0

In [375]:
# !nvprof ./K1_2_31_2_33 "in_all" "out1_255" 50 "mask_15.txt"

In [376]:
!nvprof ./K1_2_31_2_33 "in_3_4" "out1_255" 4 "mask_15.txt"

==45724== NVPROF is profiling process 45724, command: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_15.txt
==45724== Profiling application: ./K1_2_31_2_33 in_3_4 out1_255 4 mask_15.txt
==45724== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   39.90%  175.76ms         1  175.76ms  175.76ms  175.76ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int)
                   31.56%  139.03ms         1  139.03ms  139.03ms  139.03ms  [CUDA memcpy DtoH]
                   28.54%  125.74ms         2  62.869ms     736ns  125.74ms  [CUDA memcpy HtoD]
      API calls:   68.44%  441.61ms         2  220.80ms  125.99ms  315.62ms  cudaMemcpy
                   31.15%  201.00ms         2  100.50ms  368.84us  200.63ms  cudaMalloc
                    0.33%  2.1281ms         2  1.0640ms  938.59us  1.1895ms  cudaFree
                    0.05%  352.03us         1  352.03us  352.03us  352.03us  cudaMemcpyToSymbol
                    0.0

In [377]:
%%writefile K2_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <iomanip>
#include <fstream>
#include <vector>
#define STB_IMAGE_IMPLEMENTATION
#include <stdio.h>
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"
#include <string>
#include <filesystem>
#include <cuda_runtime.h>

using namespace std;
__constant__ float mask[15*15];

// Kernel definition
__global__ void Imgcov3d(int batchSize, float *input, float* output, unsigned int width,unsigned int height,int maskdim, int shared_width, int shared_height, int shared_depth){

  extern __shared__ float sharedTile [];

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int tile_x = shared_width - (maskdim - 1);
    int tile_y = shared_height - (maskdim - 1);

    int x_o = blockIdx.x * tile_x + threadIdx.x;
    int y_o = blockIdx.y * tile_y + threadIdx.y;
    int idx = blockIdx.z * blockDim.z + threadIdx.z;

    int x_i = x_o - maskdim/2;
    int y_i = y_o - maskdim/2;

    if (idx >= batchSize){
      return;
    }

  // each thread gets 1 pixel (its 3 channels) from input image
  for (int d = 0; d < 3; d++) {
    if (x_i >= 0 && y_i >= 0 && x_i < width && y_i < height){
       sharedTile[d * shared_height * shared_width + ty * shared_width + tx] =
        input[idx * width * height * 3 + d * height * width + y_i * width + x_i];
    }
    else{
       sharedTile[ d * shared_height * shared_width + ty * shared_width + tx] = 0;
    }
  }
    __syncthreads();

    // not all threads contribute in calculating the output
    if(ty < tile_y && tx < tile_x) {

      float sum = 0.0;

      for(int y_mask = 0; y_mask < maskdim; y_mask++) {
            for(int x_mask = 0; x_mask < maskdim; x_mask++) {
                  for(int z_mask = 0; z_mask < 3; z_mask++) {
                    sum += mask[ y_mask * maskdim + x_mask] *
                    sharedTile[ z_mask * shared_height * shared_width + (ty + y_mask) * shared_width + (tx + x_mask)];

                }
            }
        }

    if(sum < 0){
      sum = 0;
    } else if (sum > 255){
      sum = 255;
    }

    if(y_o < height && x_o < width)
        output[idx * width * height + y_o * width + x_o] = sum;
    }
}

int main(int argc, char *argv[]) {

    string folderPath = argv[1];
    string out_path = argv[2];
    int batch_size = stoi(argv[3]);
    string mask_path = argv[4];

    int maskdim;

    // Open the file for reading
    std::ifstream inputFile(mask_path);

    // Check if the file is opened successfully
    if (!inputFile.is_open()) {
        std::cerr << "Error opening file: " <<mask_path  << std::endl;
        return 1; // Exit with an error code
    }

     inputFile >> maskdim ;
     //cout<<"maskdim "<<maskdim<<endl;

    float *h_A, *h_B,*R,*G,*B;
    int rows1,cols1, comp;

    float *h_images;
    int img_index = 0;

    for (const auto& entry : std::filesystem::directory_iterator(folderPath)) {

        if (entry.is_regular_file()) {
            std::string filePath = entry.path().string();

            unsigned char *data = stbi_load(filePath.c_str(), &cols1, &rows1, &comp, 0);

            if(img_index == 0){
              h_images = (float*)malloc(sizeof(float) * batch_size * cols1 * rows1 * comp);
            }

            if (data) {

                /*printf("cols %d\n", cols1);
                printf("rows %d\n", rows1);
                printf("image %s\n", filePath.c_str());
                */

                h_A = (float*)malloc(sizeof(float) *  rows1 * cols1* comp);
                R = (float*)malloc(sizeof(float) *  rows1 * cols1);
                G = (float*)malloc(sizeof(float) *  rows1 * cols1);
                B = (float*)malloc(sizeof(float) *  rows1 * cols1);

                int k = 0;
                for(int i=0;i<rows1 * cols1* comp;i=i+3)
                {
                  R[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=1;i<rows1 * cols1* comp;i=i+3)
                {
                  G[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=2;i<rows1 * cols1* comp;i=i+3)
                {
                  B[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }


                memcpy(h_A, R, rows1*cols1 * sizeof(float));
                memcpy(h_A +rows1*cols1 , G, rows1*cols1 * sizeof(float));
                memcpy(h_A+rows1*cols1+rows1*cols1, B, rows1*cols1 * sizeof(float));

                memcpy(h_images + img_index * rows1 * cols1 * 3,                     R, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1,     G, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1 * 2, B, rows1 * cols1 * sizeof(float));

                img_index++;

             }
             else {
                // Failed to load the image
                std::cerr << "Failed to load image: " << filePath << std::endl;
            }

           }
      }


      float *d_images_temp = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1 * comp);
      for(int i=0; i<batch_size; i++){
        memcpy(d_images_temp + i * rows1 * cols1 *comp , h_images + i * rows1 * cols1 *comp, rows1 * cols1 *comp * sizeof(float));
      }


    float *d_images;
    cudaMalloc((void**)&d_images, batch_size * rows1 * cols1 * comp * sizeof(float));
    cudaMemcpy(d_images, d_images_temp, batch_size * rows1 * cols1 * comp * sizeof(float), cudaMemcpyHostToDevice);

    float *d_outs;
    cudaMalloc((void**)&d_outs, batch_size * rows1 * cols1 * sizeof(float));

    float *h_outs = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1);

    h_B =(float*)malloc(sizeof(float) * maskdim* maskdim);

    // Read mask elements
    for (int i = 0; i < maskdim; ++i) {
        for (int j = 0; j < maskdim; ++j) {

            int index =  i * maskdim + j;
            inputFile >>  h_B[index];

        }
    }

    // Copy data from host to device
    cudaMemcpyToSymbol(mask, h_B, maskdim * maskdim * sizeof(float), 0, cudaMemcpyHostToDevice);

    // Kernel invocation

    dim3 threadsPerBlock(16 + (maskdim - 1),16 + (maskdim - 1),1);

    int tile_x = threadsPerBlock.x - (maskdim - 1);
    int tile_y = threadsPerBlock.y - (maskdim - 1);

    int x_blocks = (cols1 + tile_x - 1) / tile_x;
    int y_blocks = (rows1 + tile_y - 1) / tile_y;
    int z_blocks = batch_size;

    int shared_width = tile_x + maskdim - 1;
    int shared_height = tile_y + maskdim - 1;
    int shared_depth = threadsPerBlock.z;

    dim3 numBlocks(x_blocks, y_blocks, z_blocks);

    Imgcov3d<<<numBlocks, threadsPerBlock,(sizeof(float)*shared_width*shared_height*3*shared_depth)>>>(batch_size,d_images,d_outs,cols1,rows1, maskdim,shared_width,shared_height,shared_depth);

    cudaMemcpy(h_outs, d_outs, batch_size * rows1 * cols1 * sizeof(float), cudaMemcpyDeviceToHost);

    unsigned char* h_char = (unsigned char*)malloc(sizeof(unsigned char) * rows1 * cols1);

    for(int k=0;k<batch_size;k++){

      for(int i=0;i<rows1*cols1;i++){
        h_char[i]=static_cast<unsigned char>(h_outs[k*rows1*cols1+i]);
      }
      string filename = "./"+out_path+"/"+"image" + std::to_string(k) + ".jpg";
      stbi_write_jpg(filename.c_str(), cols1, rows1, 1, h_char, 100);
    }

    // Free device memory
    cudaFree(d_images);
    cudaFree(d_outs);

    // Free host memory
    free(h_A);
    free(h_B);
    free(R);
    free(G);
    free(B);
    free(h_images);
    free(d_images_temp);
    free(h_outs);
    free(h_char);

  return 0;
}

Overwriting K2_2_31_2_33.cu


In [378]:
!chmod +x K2_2_31_2_33.cu
!nvcc  K2_2_31_2_33.cu -o K2_2_31_2_33

In [379]:
!nvprof ./K2_2_31_2_33 "in_3_4" "out2_255" 4 "mask_3.txt"

==45821== NVPROF is profiling process 45821, command: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_3.txt
==45821== Profiling application: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_3.txt
==45821== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   51.51%  140.38ms         1  140.38ms  140.38ms  140.38ms  [CUDA memcpy DtoH]
                   45.08%  122.86ms         2  61.429ms     672ns  122.86ms  [CUDA memcpy HtoD]
                    3.41%  9.2855ms         1  9.2855ms  9.2855ms  9.2855ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
      API calls:   59.15%  273.56ms         2  136.78ms  123.06ms  150.50ms  cudaMemcpy
                   40.25%  186.12ms         2  93.061ms  387.12us  185.73ms  cudaMalloc
                    0.46%  2.1447ms         2  1.0724ms  960.39us  1.1843ms  cudaFree
                    0.09%  424.04us         1  424.04us  424.04us  424.04us  cudaMemcpyToSymbol
          

In [380]:
!nvprof ./K2_2_31_2_33 "in_3_4" "out2_255" 4 "mask_5.txt"

==45870== NVPROF is profiling process 45870, command: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_5.txt
==45870== Profiling application: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_5.txt
==45870== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   52.85%  170.29ms         1  170.29ms  170.29ms  170.29ms  [CUDA memcpy DtoH]
                   41.62%  134.09ms         2  67.044ms     672ns  134.09ms  [CUDA memcpy HtoD]
                    5.53%  17.815ms         1  17.815ms  17.815ms  17.815ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
      API calls:   60.24%  323.45ms         2  161.73ms  134.34ms  189.11ms  cudaMemcpy
                   39.22%  210.61ms         2  105.30ms  571.19us  210.04ms  cudaMalloc
                    0.40%  2.1452ms         2  1.0726ms  950.02us  1.1951ms  cudaFree
                    0.09%  490.72us         1  490.72us  490.72us  490.72us  cudaMemcpyToSymbol
          

In [381]:
!nvprof ./K2_2_31_2_33 "in_3_4" "out2_255" 4 "mask_7.txt"

==45919== NVPROF is profiling process 45919, command: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_7.txt
==45919== Profiling application: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_7.txt
==45919== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   47.47%  139.58ms         1  139.58ms  139.58ms  139.58ms  [CUDA memcpy DtoH]
                   41.99%  123.48ms         2  61.742ms     703ns  123.48ms  [CUDA memcpy HtoD]
                   10.54%  30.994ms         1  30.994ms  30.994ms  30.994ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
      API calls:   60.78%  295.12ms         2  147.56ms  123.73ms  171.39ms  cudaMemcpy
                   38.61%  187.50ms         2  93.750ms  402.51us  187.10ms  cudaMalloc
                    0.48%  2.3382ms         2  1.1691ms  1.0147ms  1.3235ms  cudaFree
                    0.08%  410.14us         1  410.14us  410.14us  410.14us  cudaMemcpyToSymbol
          

In [382]:
!nvprof ./K2_2_31_2_33 "in_3_4" "out2_255" 4 "mask_9.txt"

==45962== NVPROF is profiling process 45962, command: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_9.txt
==45962== Profiling application: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_9.txt
==45962== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   43.17%  152.74ms         2  76.370ms     704ns  152.74ms  [CUDA memcpy HtoD]
                   38.92%  137.69ms         1  137.69ms  137.69ms  137.69ms  [CUDA memcpy DtoH]
                   17.91%  63.359ms         1  63.359ms  63.359ms  63.359ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
      API calls:   64.42%  354.84ms         2  177.42ms  152.97ms  201.87ms  cudaMemcpy
                   35.06%  193.12ms         2  96.561ms  447.88us  192.67ms  cudaMalloc
                    0.40%  2.2277ms         2  1.1139ms  1.0322ms  1.1956ms  cudaFree
                    0.08%  419.46us         1  419.46us  419.46us  419.46us  cudaMemcpyToSymbol
          

In [383]:
!nvprof ./K2_2_31_2_33 "in_3_4" "out2_255" 4 "mask_11.txt"

==46013== NVPROF is profiling process 46013, command: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_11.txt
==46013== Profiling application: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_11.txt
==46013== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   41.62%  170.36ms         1  170.36ms  170.36ms  170.36ms  [CUDA memcpy DtoH]
                   35.02%  143.33ms         2  71.666ms     704ns  143.33ms  [CUDA memcpy HtoD]
                   23.36%  95.604ms         1  95.604ms  95.604ms  95.604ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
      API calls:   65.48%  410.57ms         2  205.28ms  143.58ms  266.98ms  cudaMemcpy
                   34.05%  213.48ms         2  106.74ms  597.80us  212.88ms  cudaMalloc
                    0.34%  2.1388ms         2  1.0694ms  934.08us  1.2047ms  cudaFree
                    0.08%  481.56us         1  481.56us  481.56us  481.56us  cudaMemcpyToSymbol
        

In [384]:
!nvprof ./K2_2_31_2_33 "in_3_4" "out2_255" 4 "mask_13.txt"

==46058== NVPROF is profiling process 46058, command: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_13.txt
==46058== Profiling application: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_13.txt
==46058== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   35.56%  141.46ms         1  141.46ms  141.46ms  141.46ms  [CUDA memcpy DtoH]
                   33.11%  131.74ms         1  131.74ms  131.74ms  131.74ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
                   31.33%  124.63ms         2  62.315ms     736ns  124.63ms  [CUDA memcpy HtoD]
      API calls:   67.01%  399.22ms         2  199.61ms  124.84ms  274.38ms  cudaMemcpy
                   32.59%  194.14ms         2  97.071ms  412.34us  193.73ms  cudaMalloc
                    0.31%  1.8316ms         2  915.79us  648.73us  1.1829ms  cudaFree
                    0.06%  383.22us         1  383.22us  383.22us  383.22us  cudaMemcpyToSymbol
        

In [385]:
!nvprof ./K2_2_31_2_33 "in_3_4" "out2_255" 4 "mask_15.txt"

==46107== NVPROF is profiling process 46107, command: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_15.txt
==46107== Profiling application: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_15.txt
==46107== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   38.77%  168.51ms         1  168.51ms  168.51ms  168.51ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
                   32.87%  142.86ms         1  142.86ms  142.86ms  142.86ms  [CUDA memcpy DtoH]
                   28.37%  123.30ms         2  61.652ms     736ns  123.30ms  [CUDA memcpy HtoD]
      API calls:   68.71%  435.96ms         2  217.98ms  123.53ms  312.44ms  cudaMemcpy
                   30.84%  195.71ms         2  97.854ms  409.23us  195.30ms  cudaMalloc
                    0.35%  2.2149ms         2  1.1074ms  1.0126ms  1.2023ms  cudaFree
                    0.07%  415.69us         1  415.69us  415.69us  415.69us  cudaMemcpyToSymbol
        

In [386]:
!nvprof ./K2_2_31_2_33 "in_3_4" "out2_255" 4 "mask_edge_5.txt"

==46158== NVPROF is profiling process 46158, command: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_edge_5.txt
==46158== Profiling application: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_edge_5.txt
==46158== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   49.07%  134.91ms         1  134.91ms  134.91ms  134.91ms  [CUDA memcpy DtoH]
                   44.45%  122.21ms         2  61.104ms     672ns  122.21ms  [CUDA memcpy HtoD]
                    6.48%  17.815ms         1  17.815ms  17.815ms  17.815ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
      API calls:   58.54%  275.99ms         2  137.99ms  122.44ms  153.55ms  cudaMemcpy
                   40.86%  192.62ms         2  96.311ms  401.12us  192.22ms  cudaMalloc
                    0.49%  2.2949ms         2  1.1474ms  970.23us  1.3246ms  cudaFree
                    0.08%  377.23us         1  377.23us  377.23us  377.23us  cudaMemcpyToSymbol


In [387]:
!nvprof ./K2_2_31_2_33 "in_3_4" "out2_255" 4 "mask_3.txt"

==46211== NVPROF is profiling process 46211, command: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_3.txt
==46211== Profiling application: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_3.txt
==46211== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   53.44%  171.58ms         1  171.58ms  171.58ms  171.58ms  [CUDA memcpy DtoH]
                   43.67%  140.20ms         2  70.102ms     672ns  140.20ms  [CUDA memcpy HtoD]
                    2.89%  9.2828ms         1  9.2828ms  9.2828ms  9.2828ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
      API calls:   58.63%  322.45ms         2  161.23ms  140.43ms  182.03ms  cudaMemcpy
                   40.83%  224.55ms         2  112.28ms  602.95us  223.95ms  cudaMalloc
                    0.40%  2.1940ms         2  1.0970ms  1.0156ms  1.1784ms  cudaFree
                    0.08%  464.88us         1  464.88us  464.88us  464.88us  cudaMemcpyToSymbol
          

In [388]:
!nvprof ./K2_2_31_2_33 "in_3_4" "out2_255" 4 "mask_15.txt"

==46260== NVPROF is profiling process 46260, command: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_15.txt
==46260== Profiling application: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_15.txt
==46260== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   36.89%  150.01ms         1  150.01ms  150.01ms  150.01ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
                   32.84%  133.55ms         1  133.55ms  133.55ms  133.55ms  [CUDA memcpy DtoH]
                   30.26%  123.06ms         2  61.531ms     736ns  123.06ms  [CUDA memcpy HtoD]
      API calls:   67.59%  407.66ms         2  203.83ms  123.29ms  284.37ms  cudaMemcpy
                   31.91%  192.44ms         2  96.218ms  570.46us  191.87ms  cudaMalloc
                    0.37%  2.2476ms         2  1.1238ms  912.91us  1.3347ms  cudaFree
                    0.09%  527.41us         1  527.41us  527.41us  527.41us  cudaMemcpyToSymbol
        

In [389]:
!nvprof ./K2_2_31_2_33 "in_3_4" "out2_255" 4 "mask_15.txt"

==46309== NVPROF is profiling process 46309, command: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_15.txt
==46309== Profiling application: ./K2_2_31_2_33 in_3_4 out2_255 4 mask_15.txt
==46309== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   40.41%  178.45ms         1  178.45ms  178.45ms  178.45ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
                   31.21%  137.83ms         1  137.83ms  137.83ms  137.83ms  [CUDA memcpy DtoH]
                   28.38%  125.31ms         2  62.655ms     769ns  125.31ms  [CUDA memcpy HtoD]
      API calls:   69.34%  442.63ms         2  221.32ms  125.55ms  317.09ms  cudaMemcpy
                   30.23%  192.96ms         2  96.482ms  419.77us  192.55ms  cudaMalloc
                    0.33%  2.1301ms         2  1.0651ms  936.97us  1.1932ms  cudaFree
                    0.06%  374.24us         1  374.24us  374.24us  374.24us  cudaMemcpyToSymbol
        

In [390]:
# !nvprof ./K2_2_31_2_33 "in_all" "out2_255" 50 "mask_15.txt"

In [391]:
%%writefile K3_2_31_2_33.cu
//Norhan_Reda_Abdelwahed_2_31
//Hoda_Gamal_Hamouda_2_33

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <assert.h>
#include <iostream>
#include <iomanip>
#include <fstream>
#include <vector>
#define STB_IMAGE_IMPLEMENTATION
#include <stdio.h>
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"
#include <string>
#include <filesystem>
#include <cuda_runtime.h>

using namespace std;
__constant__ float mask[15*15];


__global__ void Imgcov3d(int batchSize, float *input, float* output, unsigned int width,unsigned int height,int maskdim, int shared_width,int shared_height, int shared_depth){

  extern __shared__ float sharedTile [];

    int tx = threadIdx.x;
    int ty = threadIdx.y;

    int tile_x = shared_width - (maskdim - 1);
    int tile_y = shared_height - (maskdim - 1);

    int x_o = blockIdx.x * tile_x + threadIdx.x;
    int y_o = blockIdx.y * tile_y + threadIdx.y;
    int idx = blockIdx.z * blockDim.z + threadIdx.z;

    if (idx >= batchSize){
      return;
    }

    // number of pixels each thread should get from input image
    int num = ((shared_width*shared_height) - 1) /(tile_x*tile_y) + 1;

    // each thread gets 'num' pixel (its 3 channels) from input image
    for (int d=0; d< 3; d++){
      for(int f=0; f < num ; f++){

          int element_index = tile_x * tile_y * f + ty * tile_x + tx;

          // get col, row in shared memory
          int dx = element_index % shared_width;
          element_index = element_index / shared_width;
          int dy = element_index % shared_height;

          // get col, row in input image
          int x_i = dx + blockIdx.x * tile_x - maskdim/2;
          int y_i = dy + blockIdx.y * tile_y - maskdim/2;

          if(x_i >= 0 && y_i >= 0 && x_i < width && y_i < height){
                sharedTile[d * shared_height * shared_width + dy * shared_width + dx]
                = input[idx * width * height * 3 + d * height * width + y_i * width + x_i];
          }
          else{
              sharedTile[d * shared_height * shared_width + dy * shared_width + dx] = 0.0;
          }
      }
    }

    __syncthreads();


  // all threads contribute in calculating the output
  if(y_o < height && x_o < width){

  float sum = 0;

    for(int y_mask = 0; y_mask < maskdim; y_mask++) {
            for(int x_mask = 0; x_mask < maskdim; x_mask++) {
                  for(int z_mask = 0; z_mask < 3; z_mask++) {
                    sum += mask[y_mask * maskdim + x_mask] *
                    sharedTile[z_mask * shared_height * shared_width + (ty + y_mask) * shared_width +(tx + x_mask)];

                }
            }
        }

    if(sum < 0){
      sum = 0;
    } else if (sum > 255){
      sum = 255;
    }

        output[idx * width * height + y_o * width + x_o] = sum;
    }
}

int main(int argc, char *argv[]) {

    string folderPath = argv[1];
    string out_path = argv[2];
    int batch_size = stoi(argv[3]);
    string mask_path = argv[4];

    int maskdim;

    // Open the file for reading
    std::ifstream inputFile(mask_path);

    // Check if the file is opened successfully
    if (!inputFile.is_open()) {
        std::cerr << "Error opening file: " <<mask_path  << std::endl;
        return 1; // Exit with an error code
    }

    inputFile >> maskdim ;
     //cout<<"maskdim "<<maskdim<<endl;

    float *h_A, *h_B,*R,*G,*B;
    int rows1,cols1, comp;

    float *h_images;
    int img_index = 0;

    for (const auto& entry : std::filesystem::directory_iterator(folderPath)) {

        if (entry.is_regular_file()) {
            std::string filePath = entry.path().string();

            unsigned char *data = stbi_load(filePath.c_str(), &cols1, &rows1, &comp, 0);

            if(img_index == 0){
              h_images = (float*)malloc(sizeof(float) * batch_size * cols1 * rows1 * comp);
            }

            if (data) {

                /*printf("cols %d\n", cols1);
                printf("rows %d\n", rows1);
                printf("image %s\n", filePath.c_str());
                */

                h_A = (float*)malloc(sizeof(float) *  rows1 * cols1* comp);
                R = (float*)malloc(sizeof(float) *  rows1 * cols1);
                G = (float*)malloc(sizeof(float) *  rows1 * cols1);
                B = (float*)malloc(sizeof(float) *  rows1 * cols1);

                int k = 0;
                for(int i=0;i<rows1 * cols1* comp;i=i+3)
                {
                  R[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=1;i<rows1 * cols1* comp;i=i+3)
                {
                  G[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }
                k=0;
                for(int i=2;i<rows1 * cols1* comp;i=i+3)
                {
                  B[k] =  static_cast<float>(data[i]);
                  k=k+1;
                }


                memcpy(h_A, R, rows1*cols1 * sizeof(float));
                memcpy(h_A +rows1*cols1 , G, rows1*cols1 * sizeof(float));
                memcpy(h_A+rows1*cols1+rows1*cols1, B, rows1*cols1 * sizeof(float));

                memcpy(h_images + img_index * rows1 * cols1 * 3,                     R, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1,     G, rows1 * cols1 * sizeof(float));
                memcpy(h_images + img_index * rows1 * cols1 * 3 + rows1 * cols1 * 2, B, rows1 * cols1 * sizeof(float));

                img_index++;

             }
             else {
                // Failed to load the image
                std::cerr << "Failed to load image: " << filePath << std::endl;
            }

           }
      }


      float *d_images_temp = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1 * comp);
      for(int i=0; i<batch_size; i++){
        memcpy(d_images_temp + i * rows1 * cols1 *comp , h_images + i * rows1 * cols1 *comp, rows1 * cols1 *comp * sizeof(float));
      }


    float *d_images;
    cudaMalloc((void**)&d_images, batch_size * rows1 * cols1 * comp * sizeof(float));
    cudaMemcpy(d_images, d_images_temp, batch_size * rows1 * cols1 * comp * sizeof(float), cudaMemcpyHostToDevice);

    float *d_outs;
    cudaMalloc((void**)&d_outs, batch_size * rows1 * cols1 * sizeof(float));

    float *h_outs = (float*)malloc(sizeof(float) * batch_size * rows1 * cols1);

    h_B =(float*)malloc(sizeof(float) * maskdim* maskdim);

    // Read mask elements
    for (int i = 0; i < maskdim; ++i) {
        for (int j = 0; j < maskdim; ++j) {

            int index =  i * maskdim + j;
            inputFile >>  h_B[index];

        }
    }

    // Copy data from host to device
    cudaMemcpyToSymbol(mask, h_B, maskdim * maskdim * sizeof(float), 0, cudaMemcpyHostToDevice);


    // Kernel invocation

    dim3 threadsPerBlock(16,16,1);

    int tile_x = threadsPerBlock.x;
    int tile_y = threadsPerBlock.y;

    int x_blocks = (cols1 + tile_x - 1) / tile_x;
    int y_blocks = (rows1 + tile_y - 1) / tile_y;
    int z_blocks = batch_size;

    int shared_width = tile_x + maskdim - 1;
    int shared_height = tile_y + maskdim - 1;
    int shared_depth = threadsPerBlock.z;

    dim3 numBlocks(x_blocks, y_blocks, z_blocks);

    Imgcov3d<<<numBlocks, threadsPerBlock,(sizeof(float)*shared_width*shared_height*3*shared_depth)>>>(batch_size,d_images,d_outs,cols1,rows1, maskdim,shared_width,shared_height,shared_depth);

    cudaMemcpy(h_outs, d_outs, batch_size * rows1 * cols1 * sizeof(float), cudaMemcpyDeviceToHost);

    unsigned char* h_char = (unsigned char*)malloc(sizeof(unsigned char) * rows1 * cols1);

    for(int k=0;k<batch_size;k++){

      for(int i=0;i<rows1*cols1;i++){
          h_char[i]=static_cast<unsigned char>(h_outs[k*rows1*cols1+i]);
      }
      string filename = "./"+out_path+"/"+"image" + std::to_string(k) + ".jpg";
      stbi_write_jpg(filename.c_str(), cols1, rows1, 1, h_char, 100);
    }

    // Free device memory
    cudaFree(d_images);
    cudaFree(d_outs);

    // Free host memory
    free(h_A);
    free(h_B);
    free(R);
    free(G);
    free(B);
    free(h_images);
    free(d_images_temp);
    free(h_outs);
    free(h_char);

  return 0;
}

Overwriting K3_2_31_2_33.cu


In [392]:
!chmod +x K3_2_31_2_33.cu
!nvcc  K3_2_31_2_33.cu -o K3_2_31_2_33

In [393]:
!nvprof ./K3_2_31_2_33 "in_3_4" "out3_255" 4 "mask_3.txt"

==46406== NVPROF is profiling process 46406, command: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_3.txt
==46406== Profiling application: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_3.txt
==46406== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   49.43%  136.52ms         1  136.52ms  136.52ms  136.52ms  [CUDA memcpy DtoH]
                   45.37%  125.32ms         2  62.658ms     704ns  125.31ms  [CUDA memcpy HtoD]
                    5.20%  14.350ms         1  14.350ms  14.350ms  14.350ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
      API calls:   58.30%  277.25ms         2  138.62ms  125.55ms  151.70ms  cudaMemcpy
                   40.89%  194.44ms         2  97.220ms  386.88us  194.05ms  cudaMalloc
                    0.67%  3.1974ms         2  1.5987ms  953.99us  2.2434ms  cudaFree
                    0.09%  427.50us         1  427.50us  427.50us  427.50us  cudaMemcpyToSymbol
          

In [394]:
!nvprof ./K3_2_31_2_33 "in_3_4" "out3_255" 4 "mask_5.txt"

==46455== NVPROF is profiling process 46455, command: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_5.txt
==46455== Profiling application: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_5.txt
==46455== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   48.28%  134.49ms         1  134.49ms  134.49ms  134.49ms  [CUDA memcpy DtoH]
                   43.37%  120.82ms         2  60.408ms     704ns  120.82ms  [CUDA memcpy HtoD]
                    8.35%  23.269ms         1  23.269ms  23.269ms  23.269ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
      API calls:   59.49%  279.59ms         2  139.80ms  121.03ms  158.56ms  cudaMemcpy
                   39.93%  187.65ms         2  93.827ms  363.14us  187.29ms  cudaMalloc
                    0.45%  2.1219ms         2  1.0610ms  916.41us  1.2055ms  cudaFree
                    0.08%  378.88us         1  378.88us  378.88us  378.88us  cudaMemcpyToSymbol
          

In [395]:
!nvprof ./K3_2_31_2_33 "in_3_4" "out3_255" 4 "mask_7.txt"

==46504== NVPROF is profiling process 46504, command: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_7.txt
==46504== Profiling application: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_7.txt
==46504== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   44.93%  137.32ms         1  137.32ms  137.32ms  137.32ms  [CUDA memcpy DtoH]
                   41.57%  127.07ms         2  63.537ms     704ns  127.07ms  [CUDA memcpy HtoD]
                   13.50%  41.267ms         1  41.267ms  41.267ms  41.267ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
      API calls:   61.96%  306.76ms         2  153.38ms  127.32ms  179.44ms  cudaMemcpy
                   37.47%  185.51ms         2  92.755ms  355.96us  185.15ms  cudaMalloc
                    0.44%  2.1885ms         2  1.0942ms  955.27us  1.2332ms  cudaFree
                    0.09%  431.26us         1  431.26us  431.26us  431.26us  cudaMemcpyToSymbol
          

In [396]:
!nvprof ./K3_2_31_2_33 "in_3_4" "out3_255" 4 "mask_9.txt"

==46549== NVPROF is profiling process 46549, command: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_9.txt
==46549== Profiling application: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_9.txt
==46549== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   41.93%  135.64ms         1  135.64ms  135.64ms  135.64ms  [CUDA memcpy DtoH]
                   37.95%  122.75ms         2  61.376ms     736ns  122.75ms  [CUDA memcpy HtoD]
                   20.12%  65.081ms         1  65.081ms  65.081ms  65.081ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
      API calls:   63.34%  324.49ms         2  162.25ms  122.96ms  201.54ms  cudaMemcpy
                   36.10%  184.92ms         2  92.458ms  360.86us  184.55ms  cudaMalloc
                    0.44%  2.2571ms         2  1.1285ms  908.27us  1.3488ms  cudaFree
                    0.08%  399.24us         1  399.24us  399.24us  399.24us  cudaMemcpyToSymbol
          

In [397]:
!nvprof ./K3_2_31_2_33 "in_3_4" "out3_255" 4 "mask_11.txt"

==46598== NVPROF is profiling process 46598, command: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_11.txt
==46598== Profiling application: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_11.txt
==46598== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   37.84%  134.63ms         1  134.63ms  134.63ms  134.63ms  [CUDA memcpy DtoH]
                   34.55%  122.93ms         2  61.463ms     704ns  122.92ms  [CUDA memcpy HtoD]
                   27.61%  98.246ms         1  98.246ms  98.246ms  98.246ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
      API calls:   65.51%  356.88ms         2  178.44ms  123.16ms  233.72ms  cudaMemcpy
                   33.79%  184.08ms         2  92.040ms  422.46us  183.66ms  cudaMalloc
                    0.59%  3.1875ms         2  1.5938ms  947.36us  2.2402ms  cudaFree
                    0.08%  448.84us         1  448.84us  448.84us  448.84us  cudaMemcpyToSymbol
        

In [398]:
!nvprof ./K3_2_31_2_33 "in_3_4" "out3_255" 4 "mask_13.txt"

==46643== NVPROF is profiling process 46643, command: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_13.txt
==46643== Profiling application: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_13.txt
==46643== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   38.80%  170.32ms         1  170.32ms  170.32ms  170.32ms  [CUDA memcpy DtoH]
                   31.09%  136.46ms         1  136.46ms  136.46ms  136.46ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
                   30.11%  132.17ms         2  66.083ms     736ns  132.16ms  [CUDA memcpy HtoD]
      API calls:   63.56%  440.50ms         2  220.25ms  132.39ms  308.11ms  cudaMemcpy
                   36.01%  249.58ms         2  124.79ms  594.20us  248.99ms  cudaMalloc
                    0.32%  2.2219ms         2  1.1109ms  925.19us  1.2967ms  cudaFree
                    0.07%  500.89us         1  500.89us  500.89us  500.89us  cudaMemcpyToSymbol
        

In [399]:
!nvprof ./K3_2_31_2_33 "in_3_4" "out3_255" 4 "mask_15.txt"

==46694== NVPROF is profiling process 46694, command: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_15.txt
==46694== Profiling application: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_15.txt
==46694== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   40.49%  174.89ms         1  174.89ms  174.89ms  174.89ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
                   31.24%  134.94ms         1  134.94ms  134.94ms  134.94ms  [CUDA memcpy DtoH]
                   28.27%  122.11ms         2  61.054ms     767ns  122.11ms  [CUDA memcpy HtoD]
      API calls:   70.18%  433.09ms         2  216.54ms  122.32ms  310.77ms  cudaMemcpy
                   29.35%  181.10ms         2  90.550ms  340.46us  180.76ms  cudaMalloc
                    0.37%  2.2644ms         2  1.1322ms  914.24us  1.3502ms  cudaFree
                    0.07%  420.97us         1  420.97us  420.97us  420.97us  cudaMemcpyToSymbol
        

In [400]:
!nvprof ./K3_2_31_2_33 "in_3_4" "out3_255" 4 "mask_edge_5.txt"

==46743== NVPROF is profiling process 46743, command: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_edge_5.txt
==46743== Profiling application: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_edge_5.txt
==46743== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   48.30%  137.35ms         1  137.35ms  137.35ms  137.35ms  [CUDA memcpy DtoH]
                   43.52%  123.76ms         2  61.882ms     672ns  123.76ms  [CUDA memcpy HtoD]
                    8.18%  23.271ms         1  23.271ms  23.271ms  23.271ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
      API calls:   60.14%  285.41ms         2  142.70ms  123.97ms  161.43ms  cudaMemcpy
                   39.24%  186.24ms         2  93.119ms  413.87us  185.82ms  cudaMalloc
                    0.48%  2.2620ms         2  1.1310ms  939.74us  1.3222ms  cudaFree
                    0.10%  468.54us         1  468.54us  468.54us  468.54us  cudaMemcpyToSymbol


In [401]:
!nvprof ./K3_2_31_2_33 "in_3_4" "out3_255" 4 "mask_3.txt"

==46790== NVPROF is profiling process 46790, command: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_3.txt
==46790== Profiling application: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_3.txt
==46790== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   49.83%  135.58ms         1  135.58ms  135.58ms  135.58ms  [CUDA memcpy DtoH]
                   44.89%  122.14ms         2  61.070ms     672ns  122.14ms  [CUDA memcpy HtoD]
                    5.27%  14.349ms         1  14.349ms  14.349ms  14.349ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
      API calls:   58.78%  273.11ms         2  136.55ms  122.38ms  150.73ms  cudaMemcpy
                   40.61%  188.69ms         2  94.343ms  347.30us  188.34ms  cudaMalloc
                    0.48%  2.2172ms         2  1.1086ms  984.86us  1.2323ms  cudaFree
                    0.08%  391.64us         1  391.64us  391.64us  391.64us  cudaMemcpyToSymbol
          

In [402]:
!nvprof ./K3_2_31_2_33 "in_3_4" "out3_255" 4 "mask_15.txt"

==46841== NVPROF is profiling process 46841, command: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_15.txt
==46841== Profiling application: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_15.txt
==46841== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   37.37%  179.62ms         1  179.62ms  179.62ms  179.62ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
                   35.45%  170.41ms         1  170.41ms  170.41ms  170.41ms  [CUDA memcpy DtoH]
                   27.18%  130.63ms         2  65.316ms     768ns  130.63ms  [CUDA memcpy HtoD]
      API calls:   67.68%  481.88ms         2  240.94ms  130.86ms  351.01ms  cudaMemcpy
                   31.91%  227.19ms         2  113.60ms  500.80us  226.69ms  cudaMalloc
                    0.30%  2.1289ms         2  1.0644ms  912.30us  1.2166ms  cudaFree
                    0.07%  484.22us         1  484.22us  484.22us  484.22us  cudaMemcpyToSymbol
        

In [403]:
!nvprof ./K3_2_31_2_33 "in_3_4" "out3_255" 4 "mask_15.txt"

==46890== NVPROF is profiling process 46890, command: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_15.txt
==46890== Profiling application: ./K3_2_31_2_33 in_3_4 out3_255 4 mask_15.txt
==46890== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   35.86%  144.48ms         1  144.48ms  144.48ms  144.48ms  Imgcov3d(int, float*, float*, unsigned int, unsigned int, int, int, int, int)
                   33.33%  134.29ms         1  134.29ms  134.29ms  134.29ms  [CUDA memcpy DtoH]
                   30.81%  124.12ms         2  62.058ms     768ns  124.12ms  [CUDA memcpy HtoD]
      API calls:   67.22%  403.91ms         2  201.95ms  124.33ms  279.58ms  cudaMemcpy
                   32.31%  194.13ms         2  97.067ms  426.00us  193.71ms  cudaMalloc
                    0.37%  2.2272ms         2  1.1136ms  908.95us  1.3182ms  cudaFree
                    0.07%  428.63us         1  428.63us  428.63us  428.63us  cudaMemcpyToSymbol
        

In [404]:
# !nvprof ./K3_2_31_2_33 "in_all" "out3_255" 50 "mask_15.txt"

In [405]:
!python pytorch_gpu.py "in_3_4" "out_py" "4" "mask_15.txt"

True
STAGE:2024-04-20 17:19:45 46936:46936 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-04-20 17:19:45 46936:46936 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-04-20 17:19:45 46936:46936 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                           aten::conv3d         0.01%      25.000us        3

In [406]:
!python pytorch.py "in_3_4" "out_py" "4" "mask_15.txt"

Folder already exists at: out_py
execution time 17.17416214942932


In [407]:
!python pytorch_gpu.py "in_3_4" "out_py" "4" "mask_13.txt"

True
STAGE:2024-04-20 17:20:17 47080:47080 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-04-20 17:20:17 47080:47080 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-04-20 17:20:17 47080:47080 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                           aten::conv3d         0.01%      25.000us        4

In [408]:
!python pytorch_gpu.py "in_3_4" "out_py" "4" "mask_11.txt"

True
STAGE:2024-04-20 17:20:24 47122:47122 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-04-20 17:20:24 47122:47122 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-04-20 17:20:24 47122:47122 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                           aten::conv3d         0.01%      23.000us        5

In [409]:
!python pytorch_gpu.py "in_3_4" "out_py" "4" "mask_9.txt"

True
STAGE:2024-04-20 17:20:32 47160:47160 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-04-20 17:20:32 47160:47160 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-04-20 17:20:32 47160:47160 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                           aten::conv3d         0.01%      23.000us        6

In [410]:
!python pytorch_gpu.py "in_3_4" "out_py" "4" "mask_7.txt"

True
STAGE:2024-04-20 17:20:39 47198:47198 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-04-20 17:20:39 47198:47198 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-04-20 17:20:39 47198:47198 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                           aten::conv3d         0.01%      22.000us        7

In [411]:
!python pytorch_gpu.py "in_3_4" "out_py" "4" "mask_5.txt"

True
STAGE:2024-04-20 17:20:46 47240:47240 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-04-20 17:20:46 47240:47240 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-04-20 17:20:46 47240:47240 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                           aten::conv3d         0.01%      23.000us        8

In [412]:
!python pytorch_gpu.py "in_3_4" "out_py" "4" "mask_3.txt"

True
STAGE:2024-04-20 17:20:54 47276:47276 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-04-20 17:20:54 47276:47276 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-04-20 17:20:54 47276:47276 ActivityProfilerController.cpp:324] Completed Stage: Post Processing
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                           aten::conv3d         0.02%      24.000us        9

In [413]:
!python pytorch.py "in_3_4" "out_py" "4" "mask_13.txt"

Folder already exists at: out_py
execution time 13.183547258377075


In [414]:
!python pytorch.py "in_3_4" "out_py" "4" "mask_11.txt"

Folder already exists at: out_py
execution time 9.925102710723877


In [415]:
!python pytorch.py "in_3_4" "out_py" "4" "mask_9.txt"

Folder already exists at: out_py
execution time 7.100457668304443


In [416]:
!python pytorch.py "in_3_4" "out_py" "4" "mask_7.txt"

Folder already exists at: out_py
execution time 5.0790863037109375


In [417]:
!python pytorch.py "in_3_4" "out_py" "4" "mask_5.txt"

Folder already exists at: out_py
execution time 2.9205591678619385


In [418]:
!python pytorch.py "in_3_4" "out_py" "4" "mask_3.txt"

Folder already exists at: out_py
execution time 1.5423247814178467
